## 網路爬蟲

### 抓標題(thread)

In [4]:
import threading
import requests
from bs4 import BeautifulSoup

# 測試網址（httpbin.org 是常見的測試網站）
urls = [
    "https://httpbin.org/html",
    "https://httpbin.org/headers",
    "https://httpbin.org/ip",
    "https://httpbin.org/user-agent",
    "https://httpbin.org/get"
]

# 工作函式：抓取網頁標題或內容
def fetch_url(url):
    try:
        response = requests.get(url, timeout=5)
        print(f"[{threading.current_thread().name}] 成功抓取：{url}")
        if "html" in response.headers.get("Content-Type", ""):
            soup = BeautifulSoup(response.text, "html.parser")
            title = soup.title.string if soup.title else "無標題"
            print(f"📝 {url} 的標題：{title}")
        else:
            print(f"🧾 {url} 的內容：{response.text[:50]}...")
    except Exception as e:
        print(f"[{threading.current_thread().name}] ⚠️ 抓取失敗：{url}\n原因：{e}")

def multi_thread_crawler(url_list):
    threads = []

    for url in url_list:
        t = threading.Thread(target=fetch_url, args=(url,))
        t.start()
        threads.append(t)

    # 等待所有執行緒結束
    for t in threads:
        t.join()

    print("✅ 所有任務完成！")

if __name__ == "__main__":
    multi_thread_crawler(urls)

[Thread-7 (fetch_url)] 成功抓取：https://httpbin.org/html
📝 https://httpbin.org/html 的標題：無標題
[Thread-10 (fetch_url)] 成功抓取：https://httpbin.org/user-agent
🧾 https://httpbin.org/user-agent 的內容：{
  "user-agent": "python-requests/2.32.4"
}
...
[Thread-9 (fetch_url)] 成功抓取：https://httpbin.org/ip
🧾 https://httpbin.org/ip 的內容：{
  "origin": "211.20.1.45"
}
...
[Thread-8 (fetch_url)] 成功抓取：https://httpbin.org/headers
🧾 https://httpbin.org/headers 的內容：{
  "headers": {
    "Accept": "*/*", 
    "Accept...
[Thread-11 (fetch_url)] 成功抓取：https://httpbin.org/get
🧾 https://httpbin.org/get 的內容：{
  "args": {}, 
  "headers": {
    "Accept": "*/*...
✅ 所有任務完成！


In [5]:
import threading
import requests
from bs4 import BeautifulSoup
from queue import Queue

# 網頁列表（可替換為你要爬的網站）
urls = [
    "https://httpbin.org/html",
    "https://httpbin.org/user-agent",
    "https://httpbin.org/headers",
    "https://httpbin.org/ip",
    "https://httpbin.org/get"
]

# 存放爬取結果
results = []

# 建立佇列
url_queue = Queue()

# 放入網址
for url in urls:
    url_queue.put(url)

# Lock 來保護共享變數（如 results）
lock = threading.Lock()

# 爬蟲執行緒函式
def crawler_worker():
    while not url_queue.empty():
        url = url_queue.get()
        try:
            response = requests.get(url, timeout=5)
            soup = BeautifulSoup(response.text, "html.parser")
            title = soup.title.string.strip() if soup.title else "無標題"
        except Exception as e:
            title = f"抓取失敗: {e}"

        with lock:
            results.append((url, title))

        url_queue.task_done()

# 建立多個執行緒
threads = []
for i in range(3):  # 可依需求調整執行緒數
    t = threading.Thread(target=crawler_worker, name=f"Worker-{i+1}")
    t.start()
    threads.append(t)

# 使用 join 等待所有執行緒完成
for t in threads:
    t.join()

# 顯示結果
print("\n📋 爬蟲結果：")
for url, title in results:
    print(f"{url} => {title}")



📋 爬蟲結果：
https://httpbin.org/headers => 無標題
https://httpbin.org/html => 無標題
https://httpbin.org/user-agent => 無標題
https://httpbin.org/ip => 無標題
https://httpbin.org/get => 無標題


### 抓標題 (lock 2)

In [6]:
import threading
import requests
from bs4 import BeautifulSoup
import time

# 建立 Semaphore，最多允許兩個執行緒同時進入
sema = threading.Semaphore(2)

urls = [
    "https://httpbin.org/html",
    "https://httpbin.org/user-agent",
    "https://httpbin.org/headers",
    "https://httpbin.org/ip",
    "https://httpbin.org/get"
]

def fetch_url(url):
    with sema:  # 取得通行證（同時最多兩個執行緒通過）
        print(f"[{threading.current_thread().name}] 正在抓取: {url}")
        try:
            response = requests.get(url, timeout=5)
            soup = BeautifulSoup(response.text, "html.parser")
            title = soup.title.string.strip() if soup.title else "無標題"
            print(f"[{threading.current_thread().name}] ✅ 成功抓取 {url} -> {title}")
        except Exception as e:
            print(f"[{threading.current_thread().name}] ❌ 錯誤: {url} -> {e}")
        time.sleep(1)  # 模擬延遲

# 建立所有爬蟲執行緒
threads = []
for url in urls:
    t = threading.Thread(target=fetch_url, args=(url,))
    t.start()
    threads.append(t)

# 等待所有執行緒完成
for t in threads:
    t.join()

print("🎉 所有抓取完成")


[Thread-12 (fetch_url)] 正在抓取: https://httpbin.org/html
[Thread-13 (fetch_url)] 正在抓取: https://httpbin.org/user-agent
[Thread-12 (fetch_url)] ✅ 成功抓取 https://httpbin.org/html -> 無標題
[Thread-13 (fetch_url)] ✅ 成功抓取 https://httpbin.org/user-agent -> 無標題
[Thread-14 (fetch_url)] 正在抓取: https://httpbin.org/headers
[Thread-15 (fetch_url)] 正在抓取: https://httpbin.org/ip
[Thread-15 (fetch_url)] ✅ 成功抓取 https://httpbin.org/ip -> 無標題
[Thread-14 (fetch_url)] ✅ 成功抓取 https://httpbin.org/headers -> 無標題
[Thread-16 (fetch_url)] 正在抓取: https://httpbin.org/get
[Thread-16 (fetch_url)] ✅ 成功抓取 https://httpbin.org/get -> 無標題
🎉 所有抓取完成


In [7]:
import threading
import requests
from bs4 import BeautifulSoup
import time

# 鎖，用於保護 counter 共享資源
lock = threading.Lock()

# 模擬同時可運行的最大執行緒數量
MAX_RUNNING = 2
current_running = 0  # 目前正在執行的執行緒數量

urls = [
    "https://httpbin.org/html",
    "https://httpbin.org/user-agent",
    "https://httpbin.org/headers",
    "https://httpbin.org/ip",
    "https://httpbin.org/get"
]

def fetch_url(url):
    global current_running

    # 等待直到有空位
    while True:
        with lock:
            if current_running < MAX_RUNNING:
                current_running += 1
                break
        time.sleep(0.1)  # 稍微等待再嘗試

    try:
        print(f"[{threading.current_thread().name}] 🚀 開始抓取: {url}")
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.text, "html.parser")
        title = soup.title.string.strip() if soup.title else "無標題"
        print(f"[{threading.current_thread().name}] ✅ 抓取完成: {url} → {title}")
        time.sleep(1)  # 模擬任務花費時間
    except Exception as e:
        print(f"[{threading.current_thread().name}] ❌ 抓取錯誤: {url} → {e}")
    finally:
        with lock:
            current_running -= 1  # 結束時釋放名額

# 建立與啟動所有執行緒
threads = []
for url in urls:
    t = threading.Thread(target=fetch_url, args=(url,))
    t.start()
    threads.append(t)

# 等待所有執行緒完成
for t in threads:
    t.join()

print("🎉 所有任務完成")


[Thread-17 (fetch_url)] 🚀 開始抓取: https://httpbin.org/html
[Thread-18 (fetch_url)] 🚀 開始抓取: https://httpbin.org/user-agent
[Thread-18 (fetch_url)] ✅ 抓取完成: https://httpbin.org/user-agent → 無標題
[Thread-17 (fetch_url)] ✅ 抓取完成: https://httpbin.org/html → 無標題
[Thread-19 (fetch_url)] 🚀 開始抓取: https://httpbin.org/headers
[Thread-21 (fetch_url)] 🚀 開始抓取: https://httpbin.org/get
[Thread-19 (fetch_url)] ✅ 抓取完成: https://httpbin.org/headers → 無標題
[Thread-21 (fetch_url)] ✅ 抓取完成: https://httpbin.org/get → 無標題
[Thread-20 (fetch_url)] 🚀 開始抓取: https://httpbin.org/ip
[Thread-20 (fetch_url)] ✅ 抓取完成: https://httpbin.org/ip → 無標題
🎉 所有任務完成


### PPT cookie 問題

In [7]:
import requests
from bs4 import BeautifulSoup

# PTT 八卦板網址（需要年齡驗證）
url = "https://www.ptt.cc/bbs/Gossiping/index.html"

# 建立 session 用來保持 Cookie
session = requests.Session()

# 模擬送出「我已滿18歲」的 Cookie
session.cookies.set("over18", "1")

# 送出帶 cookie 的 GET 請求
response = session.get(url)

# 確認是否成功通過驗證 (PTT會回應正常頁面)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    titles = soup.select(".title a")
    print("PTT 八卦板最新文章標題：")
    for t in titles[:10]:  # 只列出前10篇
        print(t.text)
else:
    print("無法存取，可能沒通過年齡驗證或網站異常")


PTT 八卦板最新文章標題：
[新聞] 自由節研討會 大紀元總裁特邀發表演講
[問卦] 擁核自重才是王道對嗎？
Re: [問卦] 日本為什麼不跟中國斷交?
Re: [新聞] 教師甄選「試教」拿20分惹議 考生痛哭2天：我錯在哪？
[新聞] 第6次！彰化「這路口」又有自小客騎上庇
Re: [新聞] 曾命中蔡英文會當總統！命理師預言柯文哲
Re: [新聞] 黃國昌挑戰司法！播放偵訊朱亞虎「示範帶
[問卦] 嘿嘿 雞湯來囉
[問卦] 詐騙集團是不是根本不用救?
[問卦] 以色列打伊朗是為了主動自衛和人道干涉？


### PPT cookie 問題 (join)

In [2]:
import threading
import requests
from bs4 import BeautifulSoup

# 要爬的 PTT 看板或頁面列表（示範幾個板）
urls = [
    "https://www.ptt.cc/bbs/Gossiping/index.html",
    "https://www.ptt.cc/bbs/Tech_Job/index.html",
    "https://www.ptt.cc/bbs/Baseball/index.html"
]

def fetch_ptt_board(url):
    # 每個執行緒都建立獨立的 session
    session = requests.Session()
    # 模擬已通過 18 歲驗證 Cookie
    session.cookies.set("over18", "1")

    try:
        res = session.get(url, timeout=10)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "html.parser")
            titles = soup.select(".title a")
            print(f"\n[{threading.current_thread().name}] {url} 最新文章標題：")
            for t in titles[:5]:  # 顯示前5篇標題
                print(" -", t.text)
        else:
            print(f"[{threading.current_thread().name}] 無法存取 {url}，HTTP狀態碼：{res.status_code}")
    except Exception as e:
        print(f"[{threading.current_thread().name}] 發生錯誤: {e}")

threads = []

# 建立並啟動執行緒
for url in urls:
    t = threading.Thread(target=fetch_ptt_board, args=(url,))
    t.start()
    threads.append(t)

# 等待所有執行緒結束
for t in threads:
    t.join()

print("\n所有板的文章標題抓取完成！")



[Thread-5 (fetch_ptt_board)] https://www.ptt.cc/bbs/Baseball/index.html 最新文章標題：
 - [新聞] MLB／紅襪Campbell遭下放3A！剛獲17.7億
 - Re: [討論] 昨日富邦跑壘
 - [分享] 山本由伸第三局11球3K
 - [新聞] 悍將下半季主題日來了！首度聯名日職關
 - [新聞] 交易後首度對決 紅襪教頭：德弗斯會想打爆

[Thread-4 (fetch_ptt_board)] https://www.ptt.cc/bbs/Tech_Job/index.html 最新文章標題：
 - [新聞] 華爾街日報：中國工程師赴第三國 租用輝達
 - [新聞] 亞馬遜執行長賈西開第一槍 示警 AI 導致
 - [討論] 哪些公司有提供免費的咖啡？
 - Re: [新聞] AI課太難 大學生爆退選停修
 - [討論] 南部是不是把主要產業全部押寶上台積了

[Thread-3 (fetch_ptt_board)] https://www.ptt.cc/bbs/Gossiping/index.html 最新文章標題：
 - [問卦] 30、40歲沒存款的是花去哪？？
 - Re: [問卦] 到底為什麼不對中共開火？？？
 - Re: [新聞] 死囚鄭武松判死定讞20年 聲請再審並停止
 - [問卦] 月薪48K管理職在高雄PR大概多少？
 - [問卦] 移工妹子都用同牌洗衣精洗髮精沐浴乳?

所有板的文章標題抓取完成！


### PPT cookie 問題 (單執行緒)

In [9]:
import threading
import requests
from bs4 import BeautifulSoup
import time

urls = [
    "https://www.ptt.cc/bbs/Gossiping/index.html",
    "https://www.ptt.cc/bbs/Tech_Job/index.html",
    "https://www.ptt.cc/bbs/Baseball/index.html"
]

def fetch_ptt_board(url):
    session = requests.Session()
    session.cookies.set("over18", "1")
    res = session.get(url, timeout=10)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "html.parser")
        titles = soup.select(".title a")
        print(f"[{threading.current_thread().name}] {url} 最新文章標題：")
        for t in titles[:3]:
            print(" -", t.text)
    else:
        print(f"[{threading.current_thread().name}] 無法存取 {url}")

def multi_thread_crawl():
    threads = []
    start = time.time()

    for url in urls:
        t = threading.Thread(target=fetch_ptt_board, args=(url,))
        t.start()
        threads.append(t)

    for t in threads:
        t.join()

    end = time.time()
    print(f"\n多執行緒花費時間: {end - start:.2f} 秒")

def single_thread_crawl():
    start = time.time()
    for url in urls:
        fetch_ptt_board(url)
    end = time.time()
    print(f"\n單執行緒花費時間: {end - start:.2f} 秒")

if __name__ == "__main__":
    print("=== 多執行緒爬取 ===")
    multi_thread_crawl()
    print("\n=== 單執行緒爬取 ===")
    single_thread_crawl()


=== 多執行緒爬取 ===
[Thread-31 (fetch_ptt_board)] https://www.ptt.cc/bbs/Gossiping/index.html 最新文章標題：
 - [問卦] 講到BMI為什麼就會有人生氣？
 - [問卦] 為什麼銀樓都喜歡收現金啊？
 - [問卦] windows 介面怎麼這麼醜?
[Thread-32 (fetch_ptt_board)] https://www.ptt.cc/bbs/Tech_Job/index.html 最新文章標題：
 - [新聞] 華爾街日報：中國工程師赴第三國 租用輝達
 - [新聞] 亞馬遜執行長賈西開第一槍 示警 AI 導致
 - [討論] 哪些公司有提供免費的咖啡？
[Thread-33 (fetch_ptt_board)] https://www.ptt.cc/bbs/Baseball/index.html 最新文章標題：
 - [情報] 道奇隊拒絕ICE探員進入球場
 - [分享] Tony Gonsolin 移往60天傷兵名單
 - [分享] 今日Logan Webb 7.0IP/1R/9K

多執行緒花費時間: 0.65 秒

=== 單執行緒爬取 ===
[MainThread] https://www.ptt.cc/bbs/Gossiping/index.html 最新文章標題：
 - [問卦] 講到BMI為什麼就會有人生氣？
 - [問卦] 為什麼銀樓都喜歡收現金啊？
 - [問卦] windows 介面怎麼這麼醜?
[MainThread] https://www.ptt.cc/bbs/Tech_Job/index.html 最新文章標題：
 - [新聞] 華爾街日報：中國工程師赴第三國 租用輝達
 - [新聞] 亞馬遜執行長賈西開第一槍 示警 AI 導致
 - [討論] 哪些公司有提供免費的咖啡？
[MainThread] https://www.ptt.cc/bbs/Baseball/index.html 最新文章標題：
 - [情報] 道奇隊拒絕ICE探員進入球場
 - [分享] Tony Gonsolin 移往60天傷兵名單
 - [分享] 今日Logan Webb 7.0IP/1R/9K

單執行緒花費時間: 1.87 秒


### PPT cookie 問題 + 內容

In [12]:
import threading
import requests
from bs4 import BeautifulSoup
import time

BASE_URL = "https://www.ptt.cc"
BOARD_URL = BASE_URL + "/bbs/Gossiping/index.html"

# 進入文章爬內文的函式，這裡同步執行，會等待此函式完成後才繼續
def fetch_article_content(url):
    session = requests.Session()
    session.cookies.set("over18", "1")
    res = session.get(url, timeout=10)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "html.parser")
        main_content = soup.find(id="main-content")
        # 移除文章內的留言區等不必要內容
        for tag in main_content.find_all(['div', 'span', 'a'], class_=['article-metaline', 'article-metaline-right', 'push']):
            tag.decompose()
        text = main_content.get_text(strip=True, separator="\n")
        print(f"\n[文章內容] {url}：\n{text[:300]}...")  # 只印前300字
    else:
        print(f"無法取得文章內容：{url}")

def fetch_board_articles():
    session = requests.Session()
    session.cookies.set("over18", "1")
    res = session.get(BOARD_URL, timeout=10)
    if res.status_code != 200:
        print("無法取得看板頁面")
        return

    soup = BeautifulSoup(res.text, "html.parser")
    articles = soup.select(".title a")

    for a in articles[:5]:  # 先抓前5篇示範
        title = a.text
        href = a['href']
        article_url = BASE_URL + href
        print(f"\n[文章標題] {title}")
        
        # 建立執行緒去爬文章內文
        t = threading.Thread(target=fetch_article_content, args=(article_url,))
        t.start()
        # 用 join 等待此文章內文爬完，再繼續下一篇
        t.join()
        print("=== 文章抓取完成 ===")

if __name__ == "__main__":
    start_time = time.time()
    fetch_board_articles()
    print(f"\n總花費時間: {time.time() - start_time:.2f} 秒")



[文章標題] [問卦] 講到BMI為什麼就會有人生氣？

[文章內容] https://www.ptt.cc/bbs/Gossiping/M.1750389849.A.B44.html：
每次講到BMI都會有人氣得跳腳

說BMI沒用 泡芙人看不出來

可是到現在醫學仍然是用BMI來判斷肥胖 過重標準

而且台灣巨巨也沒這麼多



用"體脂"來當標準的話

19歲以上成人中，過重以上人口佔全體人口之百分比
*男性體脂肪≧25%，女性體脂肪≧30%
https://i.imgur.com/SCs7smX.png
依據上述體脂率標準
男性年輕的六成過重
中老年八成

女性年輕的八成過重
中老年九成

好像比用BMI更慘


BMI是一塊遮羞布嗎

有沒有八卦？

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 220.128.198.204 (臺灣)
※ 文章網址:
...
=== 文章抓取完成 ===

[文章標題] [問卦] 為什麼銀樓都喜歡收現金啊？

[文章內容] https://www.ptt.cc/bbs/Gossiping/M.1750389850.A.A12.html：
去銀樓購買金飾的時候，想用信用卡結帳，
卻被老闆告知：
如果刷卡要被收3%，2萬要收600，
現金付款對我比較有利

為了省下小錢也是去提領了，
刷卡的回扣也比不上3%就是了

現金能讓他們逃漏多少稅啊？
5%的營業稅，還是多少營所稅？

--
※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 42.76.160.5 (臺灣)
※ 文章網址:
https://www.ptt.cc/bbs/Gossiping/M.1750389850.A.A12.html...
=== 文章抓取完成 ===

[文章標題] [問卦] windows 介面怎麼這麼醜?

[文章內容] https://www.ptt.cc/bbs/Gossiping/M.1750389864.A.EC1.html：
mac os這種的不說 畢竟果子美學還是在線的

但是隨便一個linux發行版 UBUNTU FEDORA

那個UI 字體都屌打WINDOWS

軟軟內部沒有人才嗎?設計一個好看的桌面環境很難?

為什麼WINDOWS看起來這麼醜?有無八卦?

--
如何嘴砲
反駁對方的重點──◢◣
█
確

### PPT + 看板 + 內容 (lock)

In [13]:
import threading
import requests
from bs4 import BeautifulSoup
import time
import json

BASE_URL = "https://www.ptt.cc"
BOARD_URL = BASE_URL + "/bbs/Gossiping/index.html"

results = []       # 全部文章資料會存這裡
results_lock = threading.Lock()  # 保護 results 資料結構的鎖

def fetch_article_content(url):
    session = requests.Session()
    session.cookies.set("over18", "1")
    res = session.get(url, timeout=10)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "html.parser")
        main_content = soup.find(id="main-content")
        for tag in main_content.find_all(['div', 'span', 'a'], class_=['article-metaline', 'article-metaline-right', 'push']):
            tag.decompose()
        text = main_content.get_text(strip=True, separator="\n")
        return text
    else:
        return ""

def fetch_board_articles():
    session = requests.Session()
    session.cookies.set("over18", "1")
    res = session.get(BOARD_URL, timeout=10)
    if res.status_code != 200:
        print("無法取得看板頁面")
        return

    soup = BeautifulSoup(res.text, "html.parser")
    articles = soup.select(".title a")

    for a in articles[:5]:  # 先抓前5篇示範
        title = a.text
        href = a['href']
        article_url = BASE_URL + href
        print(f"\n[文章標題] {title}")

        # 用執行緒抓文章內文並取得結果
        content = None

        def thread_job():
            nonlocal content
            content = fetch_article_content(article_url)

        t = threading.Thread(target=thread_job)
        t.start()
        t.join()

        print("=== 文章抓取完成 ===")

        # 用鎖保護 results 的修改
        with results_lock:
            results.append({
                "title": title,
                "url": article_url,
                "content": content
            })

if __name__ == "__main__":
    start_time = time.time()
    fetch_board_articles()
    print(f"\n總花費時間: {time.time() - start_time:.2f} 秒")

    # 存成 JSON 檔案
    # with open("ptt_gossiping_articles.json", "w", encoding="utf-8") as f:
    #     json.dump(results, f, ensure_ascii=False, indent=2)
    # print("資料已存成 ptt_gossiping_articles.json")



[文章標題] [問卦] 講到BMI為什麼就會有人生氣？
=== 文章抓取完成 ===

[文章標題] [問卦] 為什麼銀樓都喜歡收現金啊？
=== 文章抓取完成 ===

[文章標題] [問卦] windows 介面怎麼這麼醜?
=== 文章抓取完成 ===

[文章標題] Re: [問卦] 韓國歐巴 1992年才跟中華民國斷交？
=== 文章抓取完成 ===

[文章標題] [問卦] 中華民國臺灣那麼棒怎沒朋友？
=== 文章抓取完成 ===

總花費時間: 3.64 秒


In [ ]:
results

[{'title': '[問卦] 講到BMI為什麼就會有人生氣？',
  'url': 'https://www.ptt.cc/bbs/Gossiping/M.1750389849.A.B44.html',
  'content': '每次講到BMI都會有人氣得跳腳\n\n說BMI沒用 泡芙人看不出來\n\n可是到現在醫學仍然是用BMI來判斷肥胖 過重標準\n\n而且台灣巨巨也沒這麼多\n\n\n\n用"體脂"來當標準的話\n\n19歲以上成人中，過重以上人口佔全體人口之百分比\n*男性體脂肪≧25%，女性體脂肪≧30%\nhttps://i.imgur.com/SCs7smX.png\n依據上述體脂率標準\n男性年輕的六成過重\n中老年八成\n\n女性年輕的八成過重\n中老年九成\n\n好像比用BMI更慘\n\n\nBMI是一塊遮羞布嗎\n\n有沒有八卦？\n\n--\n※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 220.128.198.204 (臺灣)\n※ 文章網址:\nhttps://www.ptt.cc/bbs/Gossiping/M.1750389849.A.B44.html'},
 {'title': '[問卦] 為什麼銀樓都喜歡收現金啊？',
  'url': 'https://www.ptt.cc/bbs/Gossiping/M.1750389850.A.A12.html',
  'content': '去銀樓購買金飾的時候，想用信用卡結帳，\n卻被老闆告知：\n如果刷卡要被收3%，2萬要收600，\n現金付款對我比較有利\n\n為了省下小錢也是去提領了，\n刷卡的回扣也比不上3%就是了\n\n現金能讓他們逃漏多少稅啊？\n5%的營業稅，還是多少營所稅？\n\n--\n※ 發信站: 批踢踢實業坊(ptt.cc), 來自: 42.76.160.5 (臺灣)\n※ 文章網址:\nhttps://www.ptt.cc/bbs/Gossiping/M.1750389850.A.A12.html'},
 {'title': '[問卦] windows 介面怎麼這麼醜?',
  'url': 'https://www.ptt.cc/bbs/Gossiping/M.1750389864.A.EC1.html',
  'content': 'mac os

### PPT + 多看板 + 內容 (lock)

In [3]:
import threading
import requests
from bs4 import BeautifulSoup
import time
import json

BASE_URL = "https://www.ptt.cc"
BOARDS = {
    "Gossiping": "/bbs/Gossiping/index.html",
    "Baseball": "/bbs/Baseball/index.html",
    "Beauty": "/bbs/Beauty/index.html",
}

results = []
results_lock = threading.Lock()
board_lock = threading.Lock()  # 用來保護看板級別的執行流程

def fetch_article_content(url):
    session = requests.Session()
    session.cookies.set("over18", "1")
    res = session.get(url, timeout=10)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "html.parser")
        main_content = soup.find(id="main-content")
        for tag in main_content.find_all(['div', 'span', 'a'], class_=['article-metaline', 'article-metaline-right', 'push']):
            tag.decompose()
        text = main_content.get_text(strip=True, separator="\n")
        return text
    else:
        return ""

def fetch_board_articles(board_name, board_path):
    print(f"\n==== 開始抓取看板：{board_name} ====")
    session = requests.Session()
    session.cookies.set("over18", "1")
    url = BASE_URL + board_path
    res = session.get(url, timeout=10)
    if res.status_code != 200:
        print(f"無法取得看板頁面 {board_name}")
        return

    soup = BeautifulSoup(res.text, "html.parser")
    articles = soup.select(".title a")

    for a in articles[:5]:  # 先抓5篇示範
        title = a.text
        href = a['href']
        article_url = BASE_URL + href
        print(f"[{board_name}] 文章標題: {title}")

        content = None
        def thread_job():
            nonlocal content
            content = fetch_article_content(article_url)

        # 一篇文章用一個執行緒抓取文章內容
        t = threading.Thread(target=thread_job)
        t.start()
        t.join()  # 等該篇文章抓完才繼續下一篇

        print(f"[{board_name}] 文章內容抓取完成")

        with results_lock:
            results.append({
                "board": board_name,
                "title": title,
                "url": article_url,
                "content": content
            })

    print(f"==== 完成看板：{board_name} ====")

def main():
    for board_name, board_path in BOARDS.items():
        # 用board_lock確保一次只能一個看板在抓
        with board_lock:
            fetch_board_articles(board_name, board_path)

if __name__ == "__main__":
    start_time = time.time()
    main()
    print(f"\n總花費時間: {time.time() - start_time:.2f} 秒")

    # with open("ptt_multiple_boards.json", "w", encoding="utf-8") as f:
    #     json.dump(results, f, ensure_ascii=False, indent=2)

    # print("資料已存成 ptt_multiple_boards.json")



==== 開始抓取看板：Gossiping ====
[Gossiping] 文章標題: [問卦] 30、40歲沒存款的是花去哪？？
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: Re: [問卦] 到底為什麼不對中共開火？？？
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: Re: [新聞] 死囚鄭武松判死定讞20年 聲請再審並停止
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: [問卦] 月薪48K管理職在高雄PR大概多少？
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: [問卦] 移工妹子都用同牌洗衣精洗髮精沐浴乳?
[Gossiping] 文章內容抓取完成
==== 完成看板：Gossiping ====

==== 開始抓取看板：Baseball ====
[Baseball] 文章標題: [新聞] MLB／紅襪Campbell遭下放3A！剛獲17.7億
[Baseball] 文章內容抓取完成
[Baseball] 文章標題: Re: [討論] 昨日富邦跑壘
[Baseball] 文章內容抓取完成
[Baseball] 文章標題: [分享] 山本由伸第三局11球3K
[Baseball] 文章內容抓取完成
[Baseball] 文章標題: [新聞] 悍將下半季主題日來了！首度聯名日職關
[Baseball] 文章內容抓取完成
[Baseball] 文章標題: [新聞] 交易後首度對決 紅襪教頭：德弗斯會想打爆
[Baseball] 文章內容抓取完成
==== 完成看板：Baseball ====

==== 開始抓取看板：Beauty ====
[Beauty] 文章標題: [正妹] Cospaly 2044 韓國 女巫 妮姬
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 三橋くん
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 有幾顆保齡球
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 林芷芸
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 正又身材好
[Beauty] 文章內容抓取完成
==== 完成看板：Beauty

### PPT + 看板 + 內容 (單執行緒)

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import json

BASE_URL = "https://www.ptt.cc"
BOARDS = {
    "Gossiping": "/bbs/Gossiping/index.html",
    "Baseball": "/bbs/Baseball/index.html",
    "Beauty": "/bbs/Beauty/index.html",
}

results = []

def fetch_article_content(url):
    session = requests.Session()
    session.cookies.set("over18", "1")
    res = session.get(url, timeout=10)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, "html.parser")
        main_content = soup.find(id="main-content")
        for tag in main_content.find_all(['div', 'span', 'a'], class_=['article-metaline', 'article-metaline-right', 'push']):
            tag.decompose()
        text = main_content.get_text(strip=True, separator="\n")
        return text
    else:
        return ""

def fetch_board_articles(board_name, board_path):
    print(f"\n==== 開始抓取看板：{board_name} ====")
    session = requests.Session()
    session.cookies.set("over18", "1")
    url = BASE_URL + board_path
    res = session.get(url, timeout=10)
    if res.status_code != 200:
        print(f"無法取得看板頁面 {board_name}")
        return

    soup = BeautifulSoup(res.text, "html.parser")
    articles = soup.select(".title a")

    for a in articles[:5]:  # 先抓5篇示範
        title = a.text
        href = a['href']
        article_url = BASE_URL + href
        print(f"[{board_name}] 文章標題: {title}")

        # 單執行緒直接呼叫
        content = fetch_article_content(article_url)
        print(f"[{board_name}] 文章內容抓取完成")

        results.append({
            "board": board_name,
            "title": title,
            "url": article_url,
            "content": content
        })

    print(f"==== 完成看板：{board_name} ====")

def main():
    for board_name, board_path in BOARDS.items():
        fetch_board_articles(board_name, board_path)

if __name__ == "__main__":
    start_time = time.time()
    main()
    print(f"\n總花費時間: {time.time() - start_time:.2f} 秒")

    # with open("ptt_single_thread.json", "w", encoding="utf-8") as f:
    #     json.dump(results, f, ensure_ascii=False, indent=2)

    print("資料已存成 ptt_single_thread.json")



==== 開始抓取看板：Gossiping ====
[Gossiping] 文章標題: [問卦] 講到BMI為什麼就會有人生氣？
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: [問卦] 為什麼銀樓都喜歡收現金啊？
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: [問卦] windows 介面怎麼這麼醜?
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: Re: [問卦] 韓國歐巴 1992年才跟中華民國斷交？
[Gossiping] 文章內容抓取完成
[Gossiping] 文章標題: [問卦] 中華民國臺灣那麼棒怎沒朋友？
[Gossiping] 文章內容抓取完成
==== 完成看板：Gossiping ====

==== 開始抓取看板：Baseball ====
[Baseball] 文章標題: [新聞] MLB／紅襪Campbell遭下放3A！剛獲17.7億
[Baseball] 文章內容抓取完成
[Baseball] 文章標題: [公告] 板主徵選開始
[Baseball] 文章內容抓取完成
[Baseball] 文章標題: [公告] 板規 v7.0
[Baseball] 文章內容抓取完成
[Baseball] 文章標題: [整理] 2025 棒球賽事 轉播時間表
[Baseball] 文章內容抓取完成
==== 完成看板：Baseball ====

==== 開始抓取看板：Beauty ====
[Beauty] 文章標題: [正妹] Cospaly 2044 韓國 女巫 妮姬
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 三橋くん
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 有幾顆保齡球
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 林芷芸
[Beauty] 文章內容抓取完成
[Beauty] 文章標題: [正妹] 正又身材好
[Beauty] 文章內容抓取完成
==== 完成看板：Beauty ====

總花費時間: 10.58 秒
資料已存成 ptt_single_thread.json


## 資料庫

- Python 的 threading 模組特別適用於 I/O-bound 工作，像是：
    - 網路爬蟲（等待網頁回應）
    - 檔案讀寫
    - 資料庫操作
    - 等待硬碟、網路、API 等外部設備
    - 這些工作本身不會吃太多 CPU，但會讓主程式「卡住等待」，這時用多執行緒可以「一邊等，一邊做別的事情」，大幅減少總時間。
- 避免記憶體 Crash？不是主要目的，但有關係
    - 多執行緒本身並不會特別節省記憶體，反而會因為多個執行緒同時存在：
        - 佔用更多記憶體（每個 thread 有自己的堆疊）
        - 若沒管控好，可能會導致資源爭奪、死鎖、記憶體問題
> 🧯但你提到「避免記憶體 crash」這件事，是在有大量任務或不當同步時，有些人會**使用多執行緒 + 限流（像是 Semaphore）**來避免開太多資源導致崩潰，這是控制風險，不是本質目的。

| 目的                | 是否主要目的 | 補充說明                        |
| ----------------- | ------ | --------------------------- |
| ✅ 節省總執行時間         | ✅ 是    | 特別對 I/O-bound 任務            |
| ⚠️ 資源分配避免 crash   | ❌ 否    | 不是本意，但多執行緒可搭配控制機制來避免        |
| ❌ 加速 CPU-heavy 工作 | ❌ 否    | 應該用多處理程序（`multiprocessing`） |


### 同時連三個表

In [ ]:
import pymysql
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

def query_table(table_name):
    try:
        conn = pymysql.connect(
            host="HOST屏蔽",           # 加上你的 DB 主機 IP 或名稱
            user="eds",
            password="!2018Eds",       # 加上密碼
            database="sports_unify_db",
            charset="utf8mb4",
        )
        cursor = conn.cursor()
        cursor.execute(f"SELECT * FROM {table_name}")
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        df = pd.DataFrame(results, columns=columns)
        return table_name, df
    except Exception as e:
        print(f"[DataReader] Failed to read table `{table_name}`: {e}")
        return table_name, None
    finally:
        if "conn" in locals():
            conn.close()


def query_multiple_tables_parallel(table_names):
    all_results = {}

    start_time = time.time()
    with ThreadPoolExecutor(max_workers=5) as executor:  # 最多同時查詢 5 張表
        futures = [executor.submit(query_table, table) for table in table_names]

        for future in as_completed(futures):
            table_name, df = future.result()
            all_results[table_name] = df

    print(f"\n所有查詢完成，耗時：{time.time() - start_time:.2f} 秒")
    return all_results


if __name__ == "__main__":
    tables = ["ana_table", "sponsor_view", "order_lines", "orders", "league_map"]#, "match_map", "team_map"]  # 自行替換成你有的表
    # tables = ["ana_table", "sponsor_view", "order_lines", "match_map", "team_map"]
    results = query_multiple_tables_parallel(tables)

    # 範例：顯示每張表的筆數
    for table, df in results.items():
        if df is not None:
            print(f"{table}: {len(df)} rows")
        else:
            print(f"{table}: ❌ 查詢失敗")



所有查詢完成，耗時：3.77 秒
sponsor_view: 2261 rows
ana_table: 7974 rows
orders: 8225 rows
order_lines: 8094 rows
league_map: 10342 rows


### 同時連三個表 (設定匯入次數)

In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# 建立連線池引擎
engine = create_engine("mysql+pymysql://密碼屏蔽@HOST屏蔽/sports_unify_db", pool_size=100)# 最大是500

def paginated_query(table_name, batch_size=12000):
    offset = 0
    df_list = []

    try:
        while True:
            sql = f"SELECT * FROM {table_name} LIMIT {batch_size} OFFSET {offset}"
            print(f"🔍 {table_name}: 正在查詢第 {offset} 筆開始的資料...")
            try:
                df = pd.read_sql(sql, engine)
            except Exception as e:
                print(f"[查詢錯誤] {table_name} offset {offset}：{e}")
                break

            if df.empty:
                break

            df_list.append(df)
            offset += batch_size

        if df_list:
            final_df = pd.concat(df_list, ignore_index=True)
            return table_name, final_df
        else:
            return table_name, pd.DataFrame()

    except Exception as e:
        print(f"[致命錯誤] {table_name} 查詢失敗：{e}")
        return table_name, None
    
def query_multiple_tables_parallel(table_names, max_workers=4):
    all_results = {}
    start_time = time.time()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(paginated_query, table) for table in table_names]
        for future in as_completed(futures):
            table_name, df = future.result()
            all_results[table_name] = df


    print(f"\n所有查詢完成，總耗時：{time.time() - start_time:.2f} 秒")
    return all_results

if __name__ == "__main__":
    tables = ["ana_table", "sponsor_view", "order_lines", "orders", "league_map"]
    max_thread_workers = 6  # 可在這裡自由調整 worker 數
    results = query_multiple_tables_parallel(tables, max_workers=max_thread_workers)

    for table, df in results.items():
        if df is not None:
            print(f"{table}: {len(df)} rows")
        else:
            print(f"{table}: 查詢失敗")


🔍 ana_table: 正在查詢第 0 筆開始的資料...
🔍 sponsor_view: 正在查詢第 0 筆開始的資料...
🔍 order_lines: 正在查詢第 0 筆開始的資料...
🔍 orders: 正在查詢第 0 筆開始的資料...
🔍 league_map: 正在查詢第 0 筆開始的資料...
🔍 sponsor_view: 正在查詢第 12000 筆開始的資料...
🔍 ana_table: 正在查詢第 12000 筆開始的資料...
🔍 orders: 正在查詢第 12000 筆開始的資料...
🔍 order_lines: 正在查詢第 12000 筆開始的資料...
🔍 league_map: 正在查詢第 12000 筆開始的資料...

所有查詢完成，總耗時：4.08 秒
sponsor_view: 2261 rows
ana_table: 7974 rows
orders: 8225 rows
order_lines: 8094 rows
league_map: 10342 rows


In [ ]:
from sqlalchemy import create_engine
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# 建立連線池引擎
engine = create_engine("mysql+pymysql://密碼屏蔽@HOST屏蔽/sports_unify_db", pool_size=100)  # 最大是500

def paginated_query(table_name, batch_size=12000):
    offset = 0
    dfs = []  # 用 list 收集，避免反覆append造成效能問題

    try:
        while True:
            sql = f"SELECT * FROM {table_name} LIMIT {batch_size} OFFSET {offset}"
            print(f"🔍 {table_name}: 正在查詢第 {offset} 筆開始的資料...")
            try:
                df = pd.read_sql(sql, engine)
            except Exception as e:
                print(f"[查詢錯誤] {table_name} offset {offset}：{e}")
                break

            if df.empty:
                break

            dfs.append(df)
            offset += batch_size

        if dfs:
            final_df = pd.concat(dfs, ignore_index=True)
            return table_name, final_df
        else:
            return table_name, pd.DataFrame()

    except Exception as e:
        print(f"[致命錯誤] {table_name} 查詢失敗：{e}")
        return table_name, None

def query_multiple_tables_parallel(table_names, max_workers=4):
    all_results = {}
    start_time = time.time()

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(paginated_query, table) for table in table_names]
        for future in as_completed(futures):
            table_name, df = future.result()
            all_results[table_name] = df

    print(f"\n所有查詢完成，總耗時：{time.time() - start_time:.2f} 秒")
    return all_results

if __name__ == "__main__":
    tables = ["ana_table", "sponsor_view", "order_lines", "orders"]#, "league_map", "match_map", "team_map"]
    max_thread_workers = 6
    results = query_multiple_tables_parallel(tables, max_workers=max_thread_workers)

    for table, df in results.items():
        if df is not None:
            print(f"{table}: {len(df)} rows")
        else:
            print(f"{table}: 查詢失敗")


🔍 ana_table: 正在查詢第 0 筆開始的資料...
🔍 sponsor_view: 正在查詢第 0 筆開始的資料...
🔍 order_lines: 正在查詢第 0 筆開始的資料...
🔍 orders: 正在查詢第 0 筆開始的資料...
🔍 sponsor_view: 正在查詢第 12000 筆開始的資料...
🔍 ana_table: 正在查詢第 12000 筆開始的資料...
🔍 orders: 正在查詢第 12000 筆開始的資料...
🔍 order_lines: 正在查詢第 12000 筆開始的資料...

所有查詢完成，總耗時：3.48 秒
sponsor_view: 2261 rows
ana_table: 7974 rows
orders: 8225 rows
order_lines: 8094 rows
